### Задания

[0. Знакомство с данными](#0)

[1. Посчитайте, сколько книг вышло после 1 января 2000 года;](#1)

[2. Для каждой книги посчитайте количество обзоров и среднюю оценку;](#2)

[3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;](#3)

[4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;](#4)

[5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.](#5)

### Как выполнить задание?

- Опишите цели исследования;
- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результат каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.

In [12]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine


In [13]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])


In [14]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<a id="0"></a>
# Знакомство с данными

In [17]:
books = pd.io.sql.read_sql('books', con = engine)
reviews = pd.io.sql.read_sql('reviews', con = engine)
ratings = pd.io.sql.read_sql('ratings', con = engine)
authors = pd.io.sql.read_sql('authors', con = engine)
publishers = pd.io.sql.read_sql('publishers', con = engine)

In [37]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   book_id           1000 non-null   int64         
 1   author_id         1000 non-null   int64         
 2   title             1000 non-null   object        
 3   num_pages         1000 non-null   int64         
 4   publication_date  1000 non-null   datetime64[ns]
 5   publisher_id      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 47.0+ KB


In [32]:
books.head()
books.isna().sum()
books.duplicated().sum()

0

In [38]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


In [33]:
reviews.head()
reviews.isna().sum()
reviews.duplicated().sum()

0

In [39]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


In [34]:
ratings.head()
ratings.isna().sum()
ratings.duplicated().sum()

0

In [40]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


In [35]:
authors.head()
authors.isna().sum()
authors.duplicated().sum()

0

In [41]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


In [36]:
publishers.head()
publishers.isna().sum()
publishers.duplicated().sum()

0

В данных нет пропусков и дубликатов. Типы данных изменять не нужно.

<a id="1"></a>
**1. Посчитайте, сколько книг вышло после 1 января 2000 года;**

In [25]:
query = '''
SELECT 
COUNT(book_id)
FROM
books
where publication_date>'2000-01-01'
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

<a id="2"></a>
**2. Для каждой книги посчитайте количество обзоров и среднюю оценку**

In [7]:
query = '''
SELECT
AVG(ratings.rating) AS avg_rating,
ratings.book_id,
cnt_review.review_number,
books.title
FROM
ratings
INNER JOIN
(SELECT
reviews.book_id,
COUNT(review_id) AS review_number
FROM
reviews
GROUP BY
reviews.book_id) AS cnt_review ON cnt_review.book_id = ratings.book_id
INNER JOIN
books ON books.book_id = ratings.book_id
GROUP BY
ratings.book_id,
cnt_review.review_number,
books.title
ORDER BY
avg_rating DESC
'''
pd.io.sql.read_sql(query, con = engine)



,avg_rating,book_id,review_number,title
0,5.00,275,2,Geek Love
1,5.00,444,3,Moneyball: The Art of Winning an Unfair Game
2,5.00,901,2,The Walking Dead Book One (The Walking Dead #...
3,5.00,518,2,Pop Goes the Weasel (Alex Cross #5)
4,5.00,598,2,Stolen (Women of the Otherworld #2)
...,...,...,...,...
989,2.25,915,3,The World Is Flat: A Brief History of the Twen...
990,2.00,371,2,Junky
991,2.00,316,2,His Excellency: George Washington
992,2.00,202,3,Drowning Ruth


<a id="3"></a>
**3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры**

In [8]:
query = '''
SELECT
books.publisher_id,
COUNT(books.book_id) AS cnt_books,
publishers.publisher
FROM
books
INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE
num_pages>=50
GROUP BY
books.publisher_id,
publishers.publisher
ORDER BY
cnt_books DESC
'''
pd.io.sql.read_sql(query, con = engine)


,publisher_id,cnt_books,publisher
0,212,42,Penguin Books
1,309,31,Vintage
2,116,25,Grand Central Publishing
3,217,24,Penguin Classics
4,33,19,Ballantine Books
...,...,...,...
329,148,1,Harvard Business Review Press
330,302,1,Tyndale House Publishers
331,66,1,Corgi Childrens
332,230,1,Pocket Books / Simon & Schuster Inc.


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 книги.

<a id="4"></a>
**4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [9]:
query = '''
SELECT
books.author_id,
authors.author,
AVG(book_avg_rating.avg_rating) AS author_avg_rating
FROM
books
INNER JOIN (
SELECT
book_id,
AVG(rating) AS avg_rating
FROM 
ratings,
authors
GROUP BY 
book_id,
authors.author
HAVING
count(*) >= 50
) AS book_avg_rating on book_avg_rating.book_id = books.book_id
INNER JOIN authors ON authors.author_id = books.author_id
GROUP BY 
books.author_id,
authors.author
ORDER BY author_avg_rating DESC
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645
5,621,William Golding,3.901408
6,235,J.D. Salinger,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,106,Dan Brown,3.754540


<div class="alert alert-block alert-success">
<b>Комментарии от ревьюера:</b>

И здесь ты не ошибся. Ты верно отфильтровал книги и рассчитал средний рейтинг авторов
</div>

Самая высокая средняя оценка - у книг Джоан Роуллинг (4.28)

<a id="5"></a>
**5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [10]:
query = '''
SELECT
AVG(SUBQ.cnt_review)
FROM
(SELECT
username,
COUNT(review_id) AS cnt_review
FROM
reviews
WHERE 
username IN
(SELECT
username
FROM
ratings
GROUP BY
username
HAVING
COUNT(rating_id) > 50)
GROUP BY
username) AS SUBQ
'''
pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


Пользователи, которые поставили больше 50 оценок, в среднем пишут 24 обзора.